}')

# Analysis of top 10 spes by 

Find the top 10 speeches by novelty, at scale 125 and 25 and get the topics construction: top x topics and top z terms. Also include full text and labels.


In [1]:
import pandas as pd 
import numpy as np
from helpers import potus
import glob
import re
import os
from helpers import topics

importing helper functions


In [2]:
import imp
imp.reload(potus)

<module 'helpers.potus' from '/mnt/batch/tasks/shared/LS_root/mounts/clusters/ilab2-aj/code/Users/12823819/ilab2/helpers/potus.py'>

## (50t,600,10)

Load results for scales 125 and 25.

In [3]:
exp = 'potus_50t_600_10'
scales = [125, 25]
result = potus.gridExpResult(exp, scales, trim=10)

print(len(result.df))
print(len(result.settings))
display(result.settings)
display(result.df.groupby('Nw').speech.count())

load lda model from /home/azureuser/cloudfiles/code/data/processing/potus/experiment/potus_50t_600_10/lda.model
load bow from  /home/azureuser/cloudfiles/code/data/processing/potus/experiment/potus_50t_600_10/bow.pkl
File exists and is readable
/home/azureuser/cloudfiles/code/data/processing/potus/experiment/potus_50t_600_10/speeches_dfr_potus_50t_600_10_klds_100_100.pkl
/home/azureuser/cloudfiles/code/data/processing/potus/experiment/potus_50t_600_10/speeches_dfr_potus_50t_600_10_klds_125_125.pkl
t: 50, Nw: 125, Tw:125
962  before trim
/home/azureuser/cloudfiles/code/data/processing/potus/experiment/potus_50t_600_10/speeches_dfr_potus_50t_600_10_klds_25_25.pkl
t: 50, Nw: 25, Tw:25
962  before trim
/home/azureuser/cloudfiles/code/data/processing/potus/experiment/potus_50t_600_10/speeches_dfr_potus_50t_600_10_klds_50_50.pkl
/home/azureuser/cloudfiles/code/data/processing/potus/experiment/potus_50t_600_10/speeches_dfr_potus_50t_600_10_klds_75_75.pkl
1884
2


,kld_filename,topics,Nw,Tw,path
0,speeches_dfr_potus_50t_600_10_klds_125_125.pkl,50,125,125,/home/azureuser/cloudfiles/code/data/processin...
1,speeches_dfr_potus_50t_600_10_klds_25_25.pkl,50,25,25,/home/azureuser/cloudfiles/code/data/processin...


Nw
25     942
125    942
Name: speech, dtype: int64

# Top Topics and Top Terms

In [4]:
#result.model.top_topics()

## find top_novelty and top_transience

In [62]:

result.df['nr_product'] = result.df['novelty'] * result.df['resonance']

select_columns = ['date', 'title', 'speaker', 'novelty', 'transience', 'resonance', 'nr_product', 'Nw', 'topics', 'speech', 'probs']
top_novelty = result.df.groupby('Nw').apply(lambda grp: grp.nlargest(10,'novelty'))[select_columns]
top_transience = result.df.groupby('Nw').apply(lambda grp: grp.nlargest(10,'transience'))[select_columns]
top_resonance = result.df.groupby('Nw').apply(lambda grp: grp.nlargest(10,'resonance'))[select_columns]
top_product= result.df.groupby('Nw').apply(lambda grp: grp.nlargest(10,'nr_product'))[select_columns]



## query top results

```
# display top results 
idx = pd.IndexSlice
display(top_novelty.loc[idx[25,:]])
display(top_transience)
```

# topics for top results

In [6]:

for i in range(result.model.num_topics):
    terms, probs, _ = topics.print_topic(result.model,i, n=10, print_prob=False)

Topic 00: soviet soviet_union communist agreement area negotiation treaty united_nation aggression europe
Topic 01: veteran yesterday righteousness heal oklahoma wound lost child fellow_american today
Topic 02: mexico subject amount treaty revenue treasury article general territory convention
Topic 03: officer person appointment examination commission office provided rule subject clerk
Topic 04: object principle circumstance fellow_citizen view character union subject ought among
Topic 05: economy job deficit investment business income reform cost energy worker
Topic 06: panama canal isthmus water construction article city nicaragua railway zone
Topic 07: card unemployed think plan census help vice_president going unemployment health_care
Topic 08: island ship labor philippine china navy whole forest kind trade
Topic 09: legitimate_business derangement depreciated moneyed intending unproductive interior ashore overlooked depositor
Topic 10: senate treaty office secretary letter resolut

## Get topics, given text from lda_model 

```
dictionay is model.id2word 
dictionary.doc2bow(bigrams)
mode.get_document_topics(bow, min prob, per word topics = False)
```

two approachs to get the topics for a speach: 

1. use the topic probs values from dataframe, and then get the topics composition for topic probabilities 
2. start from speech, preprocess to get bigrams, convert doc2bow, get document topics on bow, then use the probabilities to print details of topic 

### Print topic probabilities from result df 

In [8]:
idx = pd.IndexSlice
speech_topic_probs = top_novelty.loc[idx[25,:]].iloc[0].probs
#speech_topic_probs


top_novelty['topics'] = top_novelty.probs.apply(lambda p: get_top_topics_from_df(p, ntopics=3, nterms=5))
top_resonance['topics'] = top_resonance.probs.apply(lambda p: get_top_topics_from_df(p, ntopics=3, nterms=5))
top_product['topics'] = top_product.probs.apply(lambda p: get_top_topics_from_df(p, ntopics=3, nterms=5))

#top_transience['topics'] = top_transience.probs.apply(lambda p: get_top_topics_from_df(p, ntopics=2, nterms=3))


#for idx, r in top_novelty.iterrows(): 
    #print(idx)
    #print(r.probs)
    
#get_top_topics_from_df(speech_topic_probs, ntopics=2)

#for topic_index in np.flip(speech_topic_probs.argsort())[:5]: 
#    terms, probs,s = get_topic(result.model,topic_index, n=10, print_prob=False)
#    print(f'{100*speech_topic_probs[topic_index]:.1f}% - {s}')

In [9]:
top_novelty[['date','title','speaker','topics', 'novelty','resonance', 'nr_product','speech']]

date                                              title  \
Nw                                                                        
25  1684  1995-04-23                          Time for Healing Ceremony   
    627   1886-05-08               Veto of Military Pension Legislation   
    416   1861-03-01  Message Regarding the Presence of Troops in Wa...   
    1485  1980-04-25               Statement on the Iran Rescue Mission   
    697   1895-06-12                       Declaration of US Neutrality   
    404   1860-05-19             Message on the Capture of the Wildfire   
    605   1882-07-01                    Veto of Safety Regulations Bill   
    1654  1993-01-29          Press Conference on Gays in the Military'   
    37    1800-05-21  Proclamation of Pardons to Those Engaged in Fr...   
    1456  1977-03-09     Remarks at President Carter's Press Conference   
125 1685  1995-04-23                          Time for Healing Ceremony   
    417   1861-03-01  Message Regarding the Presence of Troops in Wa...   
    626   1886-05-08               Veto of Military Pension Legislation   
    405   1860-05-19             Message on the Capture of the Wildfire   
    1484  1980-04-25               Statement on the Iran Rescue Mission   
    604   1882-07-01                    Veto of Safety Regulations Bill   
    696   1895-06-12                       Declaration of US Neutrality   
    36    1800-05-21  Proclamation of Pardons to Those Engaged in Fr...   
    1655  1993-01-29          Press Conference on Gays in the Military'   
    651   1889-06-04                   Statement on the Johnstown Flood   

            speaker                                             topics  \
Nw                                                                       
25  1684    clinton  [98.8% - Topic 01: veteran yesterday righteous...   
    627   cleveland  [97.7% - Topic 11: amount_public andrew presid...   
    416    buchanan  [98.3% - Topic 11: amount_public andrew presid...   
    1485     carter  [98.9% - Topic 39: party republican platform d...   
    697   cleveland  [97.3% - Topic 16: whereas president_united st...   
    404    buchanan  [99.3% - Topic 18: negro white colored race we...   
    605      arthur  [71.0% - Topic 47: space ton passenger deck to...   
    1654    clinton  [44.6% - Topic 35: dam state_texas crop run co...   
    37        adams  [84.2% - Topic 48: emancipation clay people_so...   
    1456     carter  [73.0% - Topic 26: lebanon think report vietna...   
125 1685    clinton  [98.8% - Topic 01: veteran yesterday righteous...   
    417    buchanan  [98.3% - Topic 11: amount_public andrew presid...   
    626   cleveland  [97.7% - Topic 11: amount_public andrew presid...   
    405    buchanan  [99.3% - Topic 18: negro white colored race we...   
    1484     carter  [98.9% - Topic 39: party republican platform d...   
    604      arthur  [71.0% - Topic 47: space ton passenger deck to...   
    696   cleveland  [97.3% - Topic 16: whereas president_united st...   
    36        adams  [84.2% - Topic 48: emancipation clay people_so...   
    1655    clinton  [44.6% - Topic 35: dam state_texas crop run co...   
    651   bharrison  [50.7% - Topic 06: panama canal isthmus water ...   

            novelty  resonance  nr_product  \
Nw                                           
25  1684  11.792000   0.016937    0.199724   
    627   11.182016   0.188015    2.102387   
    416   10.759741   1.118863   12.038677   
    1485   9.810816   0.723652    7.099615   
    697    8.868834   0.700112    6.209180   
    404    8.809403   0.923734    8.137543   
    605    8.650793  -0.307761   -2.662378   
    1654   7.120278   0.013430    0.095623   
    37     6.992724  -0.999089   -6.986355   
    1456   6.912694   0.275219    1.902504   
125 1685  11.708548  -0.161439   -1.890216   
    417   10.740634   0.414366    4.450551   
    626   10.361607  -0.758757   -7.861938   
    405    9.361433   1.009315    9.448630   


In [10]:
tp = top_product[['date','title','speaker','topics', 'novelty','resonance', 'nr_product','speech']].sort_values(by=['Nw','nr_product'], ascending=[True,False])
tp

date                                              title  \
Nw                                                                        
25  416   1861-03-01  Message Regarding the Presence of Troops in Wa...   
    813   1913-07-04                              Address at Gettysburg   
    1046  1941-12-08  Address to Congress Requesting a Declaration o...   
    1014  1937-10-05                                  Quarantine Speech   
    404   1860-05-19             Message on the Capture of the Wildfire   
    1027  1939-09-03              Fireside Chat 14: On the European War   
    940   1929-10-25                 Message Regarding “Black Thursday”   
    942   1929-11-05                             Message on the Economy   
    1725  1999-06-10                    Address on the Kosovo Agreement   
    1485  1980-04-25               Statement on the Iran Rescue Mission   
125 1664  1993-06-06  Speech at the 25th Anniversary Memorial Mass f...   
    405   1860-05-19             Message on the Capture of the Wildfire   
    806   1913-03-04                            First Inaugural Address   
    1134  1957-01-05                                Eisenhower Doctrine   
    1147  1960-10-21              Debate with Richard Nixon in New York   
    854   1917-04-04                           The Republic Must Awaken   
    812   1913-07-04                              Address at Gettysburg   
    1726  2000-01-27                         State of the Union Address   
    1047  1941-12-08  Address to Congress Requesting a Declaration o...   
    1764  2003-12-08            Address on Signing Medicare Legislation   

              speaker                                             topics  \
Nw                                                                         
25  416      buchanan  [98.3% - Topic 11: amount_public andrew presid...   
    813        wilson  [41.6% - Topic 24: heart democracy liberty str...   
    1046  fdroosevelt  [26.8% - Topic 49: enemy germany german berlin...   
    1014  fdroosevelt  [41.6% - Topic 00: soviet soviet_union communi...   
    404      buchanan  [99.3% - Topic 18: negro white colored race we...   
    1027  fdroosevelt  [26.1% - Topic 29: condition service case gene...   
    940        hoover  [59.6% - Topic 23: business tariff method rate...   
    942        hoover  [26.0% - Topic 23: business tariff method rate...   
    1725      clinton  [67.0% - Topic 28: iraq troop military region ...   
    1485       carter  [98.9% - Topic 39: party republican platform d...   
125 1664      clinton  [64.8% - Topic 31: applause terrorist child fa...   
    405      buchanan  [99.3% - Topic 18: negro white colored race we...   
    806        wilson  [38.1% - Topic 21: responsibility principle li...   
    1134   eisenhower  [65.9% - Topic 00: soviet soviet_union communi...   
    1147      kennedy  [26.2% - Topic 19: think administration progra...   
    854       harding  [37.1% - Topic 21: responsibility principle li...   
    812        wilson  [41.6% - Topic 24: heart democracy liberty str...   
    1726      clinton  [39.4% - Topic 32: applause child help communi...   
    1047  fdroosevelt  [26.8% - Topic 49: enemy germany german berlin...   
    1764       gwbush  [57.5% - Topic 45: health_care applause insura...   

            novelty  resonance  nr_product  \
Nw                                           
25  416   10.759741   1.118863   12.038677   
    813    4.517970   2.351272   10.622978   
    1046   4.667633   2.051148    9.574005   
    1014   4.810320   1.793473    8.627179   
    404    8.809403   0.923734    8.137543   
    1027   3.366204   2.231600    7.512020   
    940    3.678252   1.999156    7.353399   
    942    4.346593   1.659134    7.211583   
    1725   4.612292   1.548651    7.142832   
    1485   9.810816   0.723652    7.099615   
125 1664   4.624129   2.373728   10.976424   
    405    9.361433   1.009315    9.448630   
    806    4.073189   2.181010    8.883666   
  

In [69]:
tr = top_resonance[['date','title','speaker','topics', 'novelty','resonance', 'nr_product','speech']].sort_values(by=['Nw','resonance'], ascending=[True,False])

tr

date                                              title  \
Nw                                                                        
25  813   1913-07-04                              Address at Gettysburg   
    1027  1939-09-03              Fireside Chat 14: On the European War   
    1034  1940-12-29      Fireside Chat 16: On the Arsenal of Democracy   
    1046  1941-12-08  Address to Congress Requesting a Declaration o...   
    940   1929-10-25                 Message Regarding “Black Thursday”   
    807   1913-03-04                            First Inaugural Address   
    442   1865-05-02  Establishing Rewards for the Arrest of Certain...   
    1014  1937-10-05                                  Quarantine Speech   
    823   1914-05-30                               Memorial Day Address   
    427   1863-01-01                          Emancipation Proclamation   
125 1664  1993-06-06  Speech at the 25th Anniversary Memorial Mass f...   
    806   1913-03-04                            First Inaugural Address   
    854   1917-04-04                           The Republic Must Awaken   
    1726  2000-01-27                         State of the Union Address   
    812   1913-07-04                              Address at Gettysburg   
    1147  1960-10-21              Debate with Richard Nixon in New York   
    1134  1957-01-05                                Eisenhower Doctrine   
    763   1905-03-04                                  Inaugural Address   
    810   1913-06-23                   Message Regarding Banking System   
    1704  1997-01-20                                   Second Inaugural   

              speaker                                             topics  \
Nw                                                                         
25  813        wilson  [41.6% - Topic 24: heart democracy liberty str...   
    1027  fdroosevelt  [26.1% - Topic 29: condition service case gene...   
    1034  fdroosevelt  [29.0% - Topic 14: today help cannot strength ...   
    1046  fdroosevelt  [26.8% - Topic 49: enemy germany german berlin...   
    940        hoover  [59.6% - Topic 23: business tariff method rate...   
    807        wilson  [38.1% - Topic 21: responsibility principle li...   
    442       johnson  [24.3% - Topic 03: officer person appointment ...   
    1014  fdroosevelt  [41.6% - Topic 00: soviet soviet_union communi...   
    823        wilson  [47.5% - Topic 24: heart democracy liberty str...   
    427       lincoln  [29.9% - Topic 40: union constitution law texa...   
125 1664      clinton  [64.8% - Topic 31: applause terrorist child fa...   
    806        wilson  [38.1% - Topic 21: responsibility principle li...   
    854       harding  [37.1% - Topic 21: responsibility principle li...   
    1726      clinton  [39.4% - Topic 32: applause child help communi...   
    812        wilson  [41.6% - Topic 24: heart democracy liberty str...   
    1147      kennedy  [26.2% - Topic 19: think administration progra...   
    1134   eisenhower  [65.9% - Topic 00: soviet soviet_union communi...   
    763     roosevelt  [43.1% - Topic 21: responsibility principle li...   
    810        wilson  [28.1% - Topic 44: business federal problem in...   
    1704      clinton  [41.0% - Topic 12: child today family dream pr...   

           novelty  resonance  nr_product  \
Nw                                          
25  813   4.517970   2.351272   10.622978   
    1027  3.366204   2.231600    7.512020   
    1034  3.105796   2.112122    6.559818   
    1046  4.667633   2.051148    9.574005   
    940   3.678252   1.999156    7.353399   
    807   3.706513   1.846531    6.844192   
    442   3.685109   1.803011    6.644291   
    1014  4.810320   1.793473    8.627179   
    823   3.946460   1.768136    6.977880   
    427   3.659949   1.728853    6.327512   
125 1664  4.624129   2.373728   10.976424   
    806   4.073189   2.181010    8.883666   
    854   4.377506   1.763678    7.720513   
    1726  4.20003

In [76]:
lr = result.df.sort_values(by=['Nw','resonance'], ascending=[True,True])
lr.head()

,speaker,file_name,file_path,raw_tuple,title,date,speech,speech_len,probs,token_count,novelty,transience,resonance,kld_filename,topics,Nw,Tw,path,_merge,nr_product
1638,bush,bush_speeches_016.txt,/home/azureuser/cloudfiles/code/data/raw/potus...,"(Press Conference with Mikhail Gorbachev, 1991...",Press Conference with Mikhail Gorbachev,1991-07-31,"\n<President Gorbachev. Good evening, ladies a...",68705,"[0.9605945, 1.62e-06, 3.432e-05, 2.201e-05, 5....",1098,3.683658,7.749966,-4.066308,speeches_dfr_potus_50t_600_10_klds_25_25.pkl,50,25,25,/home/azureuser/cloudfiles/code/data/processin...,both,-14.978888
912,coolidge,coolidge_speeches_002.txt,/home/azureuser/cloudfiles/code/data/raw/potus...,(Centennial of Washington’s Command of the Con...,Centennial of Washington’s Command of the Cont...,1925-07-03,"After 150 anniversaries repeatedly observed, f...",37455,"[3.803e-05, 2.67e-06, 0.04789578, 3.625e-05, 0...",522,2.643174,4.913795,-2.270621,speeches_dfr_potus_50t_600_10_klds_25_25.pkl,50,25,25,/home/azureuser/cloudfiles/code/data/processin...,both,-6.001646
1094,truman,truman_speeches_005.txt,/home/azureuser/cloudfiles/code/data/raw/potus...,"(Announcing the Surrender of Japan, 1945-09-01...",Announcing the Surrender of Japan,1945-09-01,The thoughts and hopes of all America -- indee...,6950,"[0.02279478, 1.56e-05, 0.00033028, 0.0002118, ...",1195,1.450779,3.612667,-2.161888,speeches_dfr_potus_50t_600_10_klds_25_25.pkl,50,25,25,/home/azureuser/cloudfiles/code/data/processin...,both,-3.136422
1084,fdroosevelt,fdroosevelt_speeches_000.txt,/home/azureuser/cloudfiles/code/data/raw/potus...,"(Address to Congress on Yalta, 1945-03-01, I h...",Address to Congress on Yalta,1945-03-01,I hope that you will pardon me for this unusua...,34295,"[0.06290758, 3.02e-06, 6.403e-05, 4.104e-05, 0...",1797,1.379677,3.368356,-1.988679,speeches_dfr_potus_50t_600_10_klds_25_25.pkl,50,25,25,/home/azureuser/cloudfiles/code/data/processin...,both,-2.743735
1092,truman,truman_speeches_004.txt,/home/azureuser/cloudfiles/code/data/raw/potus...,(Radio Report to the American People on the Po...,Radio Report to the American People on the Pot...,1945-08-09,My fellow Americans:\nI have just returned fro...,23464,"[0.1088208, 4.28e-06, 0.03044812, 5.817e-05, 0...",1753,1.562333,3.541521,-1.979188,speeches_dfr_potus_50t_600_10_klds_25_25.pkl,50,25,25,/home/azureuser/cloudfiles/code/data/processin...,both,-3.092151


In [82]:
lr.iloc[1]['title'], lr.iloc[1]['speech']

('Centennial of Washington’s Command of the Continental Army',
 'After 150 anniversaries repeatedly observed, followed during the last three months by intensive celebration, in this neighborhood where it had its beginnings, the American Revolution should be fairly well understood. If it needs any justification, if it needs any praise, it is enough to say that its product is America. It ought to be unnecessary on this occasion to dwell very much on that event and its yet more remarkable results. But no great movement in the progress of mankind has ever been accomplished without the guidance of an inspired leadership. Of this accepted truth, there is no more preeminent example than that which was revealed by the war which made this country independent. Wherever men love liberty, wherever they believe in patriotism, wherever they exalt high character, by universal consent they turn to the name of George Washington. No occasion could be conceived more worthy, more truly and comprehensively

In [72]:
# First by resonance; the 125 speech window

speeches_to_read = []
for idx,row in tr[:6].iterrows():
    topics = "; ".join(row['topics'])  
    speeches_to_read.append((row['title'], row['novelty'], row['resonance'], row['speaker'], row['date'], topics, row['speech']))
    
speeches_to_read

[('Address at Gettysburg',
  4.517970085144043,
  2.3512723445892334,
  'wilson',
  datetime.date(1913, 7, 4),
  '41.6% - Topic 24: heart democracy liberty struggle washington; 21.9% - Topic 21: responsibility principle liberty justice human; 12.9% - Topic 29: condition service case general line',
  "Friends and Fellow Citizens:\nI need not tell you what the Battle of Gettysburg meant. These gallant men in blue and gray sit all about us here. Many of them met upon this ground in grim and deadly struggle. Upon these famous fields and hillsides their comrades died about them. In their presence it were an impertinence to discourse upon how the battle went, how it ended, what it signified! But fifty years have gone by since then, and I crave the privilege of speaking to you for a few minutes of what those fifty years have meant.\nWhat have they meant? They have meant peace and union and vigor, and the maturity and might of a great nation. How wholesome and healing the peace has been! We ha

In [51]:
# Second 10; the 125 speech window

speeches_to_read = []
for idx,row in tr[10:].iterrows():
    topics = "; ".join(row['topics'])  
    speeches_to_read.append((row['title'], row['resonance'], row['speaker'], row['date'], topics, row['speech']))
    
speeches_to_read

[('Speech at the 25th Anniversary Memorial Mass for Robert F. Kennedy',
  2.373728036880493,
  'clinton',
  datetime.date(1993, 6, 6),
  '64.8% - Topic 31: applause terrorist child family today; 19.3% - Topic 12: child today family dream promise; 9.8% - Topic 27: going think thing back tell',
  'Father Creedon, Mrs. Kennedy, the children of Robert Kennedy, and the Kennedy family, to all the distinguished Americans here present, and most of all, to all of you who bear the noble title, citizen of this country:\nTwenty-five years ago today, on the eve of my college graduation, I cheered the victory of Robert Kennedy in the California primary and felt again that our country might face its problems openly, meet its challenges bravely, and go forward together. He dared us all. He dared the grieving not to retreat into despair. He dared the comfortable not to be complacent. He dared the doubting to keep going.\nAs I looked around this crowd today and saw us all graced not only by the laughter

### get topic probabiilties from speech text

In [11]:
import numpy as np

### firt need to convert text to bigrams -- TBD

bigrams = ['united_states', 'space', 'vessel']

bow1 = result.model.id2word.doc2bow(bigrams)
#bow1 = result.model.id2word.doc2bow(['lebanon', 'middleeast'])
#bow1 = result.model.id2word.doc2bow(['medicare', 'america', 'vetran'])
bow1 = result.model.id2word.doc2bow(['war', 'kuwait'])
doc_topics = result.model.get_document_topics(bow1)

topic_numbers = np.array([r[0] for r in doc_topics])
topic_probabilities = np.array([r[1] for r in doc_topics])

for ti in np.flip(topic_probabilities.argsort()): 
    terms, probs,s = topics.get_topic(result.model,topic_numbers[ti], n=10, print_prob=False)
    print(f'{100*topic_probabilities[ti]:.1f}% - {s}')

39.8% - Topic 28: iraq troop military region israel attack afghanistan iraqi middle_east regime
9.8% - Topic 29: condition service case general line officer army subject result regard
4.1% - Topic 14: today help cannot strength tonight program goal energy defense problem
3.6% - Topic 04: object principle circumstance fellow_citizen view character union subject ought among
3.1% - Topic 21: responsibility principle liberty justice human task thing wish seek civilization
3.1% - Topic 44: business federal problem individual increase program administration industry legislation service
3.1% - Topic 24: heart democracy liberty struggle washington speak word revolution honor victory
2.4% - Topic 40: union constitution law texas authority constitutional rebellion act military executive
2.4% - Topic 02: mexico subject amount treaty revenue treasury article general territory convention
2.3% - Topic 27: going think thing back tell something done look issue little
1.9% - Topic 12: child today famil

# All Topics 

In [54]:
topicid = 1
n=15
lda_model = result.model
print_prob = False

topics_df = pd.DataFrame()


for topicid in range(0,lda_model.num_topics):
    topic_terms = lda_model.get_topic_terms(topicid, topn=n)
    
    terms = list()
    probs = list()
    for termid, prob in topic_terms:
        term = lda_model.id2word.get(termid)
        terms.append(term)
        probs.append(prob)
        
    topics_df = topics_df.append(pd.DataFrame({'topic': topicid, 'terms': terms, 'probs': probs}).reset_index().rename(columns={'index':'order'}))

topics_df.reset_index(drop=True)

topics_df.to_csv('./data/topics_50t_600_10.csv')

In [50]:
15*50


750